<a id="top"></a>
# Goal: Identify all EPA facilities related to incarceration.

### Background
EPA 'facilities' (sites or places) subject to environmental regulation are identified across programs in various databases by a REGISTRY_ID.

This notebook explores how we might identify all facilities related to incarceration, to get a list of REGISTRY_ID for use in querying other data sources.

### Data Source
Facility data are maintained by the EPA's [Facility Registry Services (FRS)](https://www.epa.gov/frs).

This notebook explores the [national 'single file CSV'](https://www.epa.gov/frs/epa-frs-facilities-state-single-file-csv-download) download of FRS facility data on 4.1 million facilities, an export from this larger [facility data model](https://www.epa.gov/frs/frs-sub-facility-data-model).

### Recommendations and Summary

#### Recommendations
- we cannot programatically ID all incarceration facilities from these 4.1 million records, using this dataset's features
- we'll likely need to combine programmatic approaches with manual review/coding
- a list of known incarceration sites (such as [this one from Homeland Security](https://hifld-geoplatform.opendata.arcgis.com/datasets/2d6109d4127d458eaf0958e4c5296b67_0)) may be useful as a starting point
- another approach is to start from the FRS data here and use reasonable search criteria to ID as many carceral facilities as possible, without matching to a known list (update: we've since learned that in fact many prisons are not registered with the EPA)
- even with a list of known incarceration sites, EPA 'facilities' may include sites and places beyond the 'storefront' locations with recognizable names (e.g. underwater tanks, laundry facilities)


#### Summary
Based on the available data fields, this notebook explores some possible ways to ID incarceration facilities:

(hyperlinks go to corresponding notebook section)

- [NAICS/SIC codes](#industrycodes)
    - 66% of 4.1 million records have no NAICS or SIC code
    - Some incarceration-related facilities are coded with incorrect or nonintuitive NAICS/SIC codes
    - 1,527 records are coded as 'CORRECTIONAL INSTITUTIONS'


- [Facility name (string matching)](#stringmatching)
	- (e.g. ID record if name has 'CORRECTIONAL' or 'JAIL' etc.)
	- liberal match strings are needed for sensitivity, increasing false positives
		- e.g. Facility name 'CCI' for California Correctional Institution, returns 'VACCINATION' etc.
	- many facilities use unintuitve abbreviations
	- a strict list of match strings ID'd 2,845 facilities not coded by NAICS/SIC (still some false positives)
    - (**update**: our group performed preliminary probabilistic fuzzy matching in R via RecordLinkage, between ECHO/FRS data and [HIFLD](https://hifld-geoplatform.opendata.arcgis.com/datasets/2d6109d4127d458eaf0958e4c5296b67_0), using several fields (including name); results still required manual coding, likely suffer from many prisons just not being registered with EPA)


- [Location: latitude/longitude](#latlong)
    - (based on offline discussions of spatial joins with a non-EPA source of incarceration facility GIS boundaries)
    - 28% of records have no lat/long
    - long/lat records are collected by methods of varying accuracy, including:
        - street address, zip code centroid, unknown
    - looking at long/lat records for NAICS/SIC coded incarceration facilities, we still find long/lat methods of varying accuracy
    - (**update**: I performed a preliminary spatial join in R using FRS and [HIFLD](https://hifld-geoplatform.opendata.arcgis.com/datasets/2d6109d4127d458eaf0958e4c5296b67_0) geodatabase files; results still required manual coding, likely suffer from many prisons just not being registered with EPA)


- [Location: address](#address)
    - street address matching in general is fraught
    - 11% of records have no street address (missing)
    - street address values vary in quality


- Other fields (not fully explored here)
    - (unlikely to yield much, maybe to eliminate records)
    - EPA program type ('INTEREST_TYPES')
    - Site type name ('SITE_TYPE_NAME')

## Setup Notebook

In [1]:
import pandas as pd
import numpy as np

In [2]:
#our dataset has 39 columns, display all if desired
pd.set_option('display.max_columns', None)

#display all output not just last
#https://stackoverflow.com/questions/36786722/how-to-display-full-output-in-jupyter-not-only-last-result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Load Data, Inspect Columns
There are 4,147,930 records and 39 columns.

Column descriptions are included in the [data source](https://www.epa.gov/frs/epa-frs-facilities-state-single-file-csv-download) zip file as PDF documentation.

In [3]:
fac = pd.read_csv('NATIONAL_SINGLE.CSV')

C:\Users\Ben\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,5,13,14,15,16,21,24,27,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
fac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4147930 entries, 0 to 4147929
Data columns (total 39 columns):
FRS_FACILITY_DETAIL_REPORT_URL    object
REGISTRY_ID                       int64
PRIMARY_NAME                      object
LOCATION_ADDRESS                  object
SUPPLEMENTAL_LOCATION             object
CITY_NAME                         object
COUNTY_NAME                       object
FIPS_CODE                         object
STATE_CODE                        object
STATE_NAME                        object
COUNTRY_NAME                      object
POSTAL_CODE                       object
FEDERAL_FACILITY_CODE             object
FEDERAL_AGENCY_NAME               object
TRIBAL_LAND_CODE                  object
TRIBAL_LAND_NAME                  object
CONGRESSIONAL_DIST_NUM            object
CENSUS_BLOCK_CODE                 float64
HUC_CODE                          float64
EPA_REGION_CODE                   float64
SITE_TYPE_NAME                    object
LOCATION_DESCRIPTION    

Confirming REGISTRY_ID is a unique key (REGISTRY_ID is not unique in the [FRS geodatabase files](https://www.epa.gov/frs/geospatial-data-download-service) used subsequently for a spatial join); where instead the key was REGISTRY_ID + statute):

In [5]:
#confirm Registry IDs are unique
sum(fac['REGISTRY_ID'].duplicated())

0

<a id="industrycodes"></a>
## Using NAICS/SIC codes to ID facilities related to incarceration
### Notes
    - here are some examples of facilities NAICS/SIC coded as Correctional Institutions but with uninformative/unintuitive names: AUBURN RESIDENTIAL CENTER, COUNTY OF CUMBERLAND, NJDOC ADMIN OFFICES, CCI TEHACHAPI, FIRE CAMP #11NA ACTON, UNKNOWN, 3600 GUARD ROAD

<a href="#top">Back to Top of Notebook</a>

##### Inspect NAICS / SIC codes, how many missing values?
We find 66% of our 4+ million records have no NAICS or SIC code. Some records are assigned more than one code.

In [35]:
fac['NAICS_CODES'].value_counts(dropna=False).head(10)
fac['NAICS_CODES'].value_counts(dropna=False).tail(10)

NaN       3291831
211111      35958
811111      31210
811121      24572
812320      18737
447110      18308
221122      12471
211112      12221
611110      10528
562212      10137
Name: NAICS_CODES, dtype: int64

424710, 611310                                                    1
321910, 321918, 337110, 337122                                    1
023835, 337110                                                    1
336320, 336370                                                    1
811113, 811310                                                    1
111998, 713110                                                    1
211111, 211120, 213111, 213112, 486910                            1
044111, 111331                                                    1
311220, 332321, 812320                                            1
211111, 211120, 211130, 213111, 213112, 221320, 237120, 424710    1
Name: NAICS_CODES, dtype: int64

In [36]:
fac['SIC_CODES'].value_counts(dropna=False).head(10)
fac['SIC_CODES'].value_counts(dropna=False).tail(10)

NaN     3108299
1311      52920
9999      49754
5541      47051
4952      41323
1629      28699
1521      18662
7216      18440
1611      17745
7532      14439
Name: SIC_CODES, dtype: int64

3341, 3399, 7389                                        1
2851, 2869, 2899, 2944, 4225                            1
0211, 0241, 1442, 3625, 4011, 4226, 4911, 4952, 5541    1
0711, 0782                                              1
4899, 7376                                              1
2834, 2842, 2899, 5122                                  1
3061, 3069, 3949                                        1
4212, 4225, 7513                                        1
2281, 2282, 2822, 9999                                  1
3471, 3714, 4225                                        1
Name: SIC_CODES, dtype: int64

In [4]:
missing_naics_and_sic = pd.isna(fac['NAICS_CODES']) & pd.isna(fac['SIC_CODES'])
sum(missing_naics_and_sic)/fac.shape[0]

0.6606644760157476

##### How many records have the NAICS / SIC codes for 'CORRECTIONAL INSTITUTION'?
1,527 records are coded as CORRECTIONAL INSTITUTIONS with NAICS code (922140) or SIC code (9223).

In [5]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html
#not exact, returns more results, need inspect for false positives
naics_correctional = fac['NAICS_CODES'].str.contains('922140', na=False, regex=False) #a logical Series
sum(naics_correctional)

982

In [6]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html
#not exact, returns more results, need inspect for false positives
sic_correctional = fac['SIC_CODES'].str.contains('9223', na=False, regex=False) #a logical Series
sum(sic_correctional)

1274

In [7]:
naics_or_sic_correctional = naics_correctional | sic_correctional
sum(naics_or_sic_correctional)

1527

In [9]:
#Column subset to inspect, for convenienct reuse
industry_code_columns = ['REGISTRY_ID','PRIMARY_NAME','STATE_NAME','NAICS_CODES','NAICS_CODE_DESCRIPTIONS','SIC_CODES','SIC_CODE_DESCRIPTIONS']
fac[naics_or_sic_correctional].sample(n=10)[industry_code_columns]

,REGISTRY_ID,PRIMARY_NAME,STATE_NAME,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS
612543,110055834628,LASSEN COUNTY JUVENILE DETENTION FACILITY,CALIFORNIA,922150,PAROLE OFFICES AND PROBATION OFFICES.,9223,CORRECTIONAL INSTITUTIONS
1239863,110018362831,METROPOLITAN CORRECTIONAL CTR,ILLINOIS,NaN,NaN,9223,CORRECTIONAL INSTITUTIONS
1788892,110019857760,DPSCS/DCCFM - CENTRAL LAUNDRY FACILITY,MARYLAND,"922140, 922190","CORRECTIONAL INSTITUTIONS., OTHER JUSTICE, PUB...","7211, 9223","CORRECTIONAL INSTITUTIONS, POWER LAUNDRIES, FA..."
3907985,110017286257,VDOC - BLAND CORRECTIONAL CENTER,VIRGINIA,922140,CORRECTIONAL INSTITUTIONS.,"4952, 9223, OWNE","CORRECTIONAL INSTITUTIONS, SEWERAGE SYSTEMS"
1580201,110044952548,LFUCG - FAYETTE CO DETENTION CENTER,KENTUCKY,NaN,NaN,9223,CORRECTIONAL INSTITUTIONS
3929756,110008201659,DEEP MEADOW CORRECTIONAL CENTER,VIRGINIA,NaN,NaN,"4911, 9223, OWNE","CORRECTIONAL INSTITUTIONS, ELECTRIC SERVICES"
886214,110005597755,OSCEOLA COUNTY,FLORIDA,922140,CORRECTIONAL INSTITUTIONS.,9223,CORRECTIONAL INSTITUTIONS
270167,110002811994,CALIF. DEPT OF CORRECTIONS - PELICAN BAY PRISON,CALIFORNIA,922140,CORRECTIONAL INSTITUTIONS.,NaN,NaN
4103894,110024429698,UNITED STATE PENITENTIARY - HAZELTON,WEST VIRGINIA,922140,CORRECTIONAL INSTITUTIONS.,9223,CORRECTIONAL INSTITUTIONS
782781,110001732153,RADGOWSKI/CORRIGAN CORRECTIONAL INSTITUTE,CONNECTICUT,922140,CORRECTIONAL INSTITUTIONS.,"9223, OWNE",CORRECTIONAL INSTITUTIONS


##### How many records have NAICS / SIC that are something other than 'CORRECTIONAL INSTITUTION'?

1,406,013.
Justification: in case we wanted to filter out records coded as not correctional (below in 'string matching' section we'll find some other NAICS/SIC coded records appear to be incarceration, so not 100% reliable).

In [12]:
not_correctional = ~missing_naics_and_sic & ~naics_or_sic_correctional
fac[not_correctional].shape
fac[not_correctional].sample(5)[industry_code_columns]

(1406013, 39)

,REGISTRY_ID,PRIMARY_NAME,STATE_NAME,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS
215216,110002915123,THE VONS COMPANIES INC,CALIFORNIA,445110,SUPERMARKETS AND OTHER GROCERY (EXCEPT CONVENI...,5411,GROCERY STORES
2197604,110042457436,ACRES OF SHADE MHP,MISSOURI,NaN,NaN,6515,OPERATORS OF RESIDENTIAL MOBILE HOME SITES
408070,110070452050,CISCO SYSTEMS INC- SYCAMORE II,CALIFORNIA,"334418, 334613",MAGNETIC AND OPTICAL RECORDING MEDIA MANUFACTU...,NaN,NaN
3391334,110010260369,RICHARD AND MARY SANDERS,PENNSYLVANIA,NaN,NaN,6514,OPERATORS OF DWELLINGS OTHER THAN APARTMENT BU...
4004743,110015485768,SMITHS AUTO SERVICE,WASHINGTON,"008111, 447190",OTHER GASOLINE STATIONS.,0753,NaN


<a id="stringmatching"></a>
## Using string matching (e.g. 'PRISON') in Facility Name to ID facilities

<a href="#top">Back to Top of Notebook</a>

##### Can we use string matching into ID facilities (e.g. 'PRISON')?

Proof of concept, we find 839 records w/ 'CORRECTIONAL' in the facility name, that aren't coded with NAICS/SIC:

In [54]:
string_in_name = fac['PRIMARY_NAME'].str.contains('CORRECTIONAL', na=False, regex=False) #a logical Series, #tilde here is bitwise not
fac[string_in_name & ~naics_or_sic_correctional].shape
fac[string_in_name & ~naics_or_sic_correctional].sample(n=5)[industry_code_columns] #~ is bitwise complement operator, using with numpy boolean Series, in general use caution

(839, 39)

,REGISTRY_ID,PRIMARY_NAME,STATE_NAME,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS
2929917,110070516105,DIVERSEY GREAT MEADOW CORRECTIONAL FACILITY,NEW YORK,NaN,NaN,NaN,NaN
4097021,110012854535,THOMPSON CORRECTIONAL CENTER,WISCONSIN,NaN,NaN,NaN,NaN
934643,110024385761,TAYLOR CORRECTIONAL INSTITUTE,FLORIDA,NaN,NaN,NaN,NaN
1068364,110007490893,GA CORRECTIONAL IND METAL FAB PLT,GEORGIA,NaN,NaN,NaN,NaN
3952277,110068403392,SE CORRECTIONAL INSTITUTE,VERMONT,NaN,NaN,NaN,NaN


Let's try with an expanded list of strings to match.

We gathered the expanded list manually from looking at NAICS/SIC coded records; at first the list was too permissive e.g. 'CCI' (California Correctional Institution) returned 'VACCINATION' etc.

In [14]:
#https://stackoverflow.com/questions/26577516/how-to-test-if-a-string-contains-one-of-the-substrings-in-a-list-in-pandas
# the first 'searchfor' list was too permissive e.g. 'DOC' returned DOCK, too many false positives
#searchfor = ['PRISON','CORR','DETENTION','JAIL','JUSTICE','REHAB','PENITENTIARY','SHERIFF','JUVENILE','CCI','COLONY','MCI','TDCJ','SCI','DOC']
searchfor = ['PRISON','CORRECTIONAL','DETENTION','JAIL','PENITENTIARY']
strings_in_name = fac['PRIMARY_NAME'].str.contains('|'.join(searchfor), na=False, regex=True)

In [48]:
fac.shape #just for comparison
fac[strings_in_name & ~naics_or_sic_correctional].shape
fac[strings_in_name & ~naics_or_sic_correctional].sample(n=5)[industry_code_columns]

(4147930, 39)

(2845, 39)

,REGISTRY_ID,PRIMARY_NAME,STATE_NAME,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS
3364251,110070264720,WOMENS CORRECTIONAL FACILITY,PENNSYLVANIA,NaN,NaN,NaN,NaN
3600514,110027680668,GILES COUNTY JAIL,TENNESSEE,NaN,NaN,NaN,NaN
1993423,110068048856,R R SITE - DETENTION POND - CSW,MINNESOTA,NaN,NaN,NaN,NaN
3536729,110048086441,CO PRISON FARM,SOUTH CAROLINA,NaN,NaN,NaN,NaN
4111819,110010883464,DENMAR CORRECTIONAL CTR,WEST VIRGINIA,NaN,NaN,8062,GENERAL MEDICAL AND SURGICAL HOSPITALS


Here by chance we find a false positive `WEYMOUTH ANIMAL DETENTION CENTER`:

(note: users running this notebook on their machine will get different random records from the 'sample' method)

In [53]:
fac[strings_in_name & ~naics_or_sic_correctional].sample(n=5)[industry_code_columns]

,REGISTRY_ID,PRIMARY_NAME,STATE_NAME,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS
1591823,110069224841,ROWAN COUNTY DETENTION CENTER,KENTUCKY,237310,"HIGHWAY, STREET, AND BRIDGE CONSTRUCTION.",1611,"HIGHWAY AND STREET CONSTRUCTION, EXCEPT ELEVAT..."
1747676,110001997858,WEYMOUTH ANIMAL DETENTION CENTER,MASSACHUSETTS,812910,PET CARE (EXCEPT VETERINARY) SERVICES.,"0752, OWNE","ANIMAL SPECIALTY SERVICES, EXCEPT VETERINARY"
1097074,110013128864,SCREVEN CO. PRISON,GEORGIA,NaN,NaN,NaN,NaN
1412996,110070113760,WALMART #6602 - DETENTION POND REHABILTION,INDIANA,NaN,NaN,NaN,NaN
183848,110039295125,PIMA COUNTY - CORRECTIONAL FACILITY,ARIZONA,NaN,NaN,NaN,NaN


##### Are there any incarceration facilities <u>incorrectly</u> coded with NAICS/SIC?
*Possibly*... using string matching, we find facilities that have names of interest (`LAKE ERIE CORRECTIONAL CENTER`) but perhaps unintuitive SIC_CODE_DESCRIPTION (`GASOLINE SERVICE STATIONS`).

In [56]:
#match string must be in name, must have NAICS and/or SIC, and must not be coded as CORRECTIONAL INSTITUTION
fac[(strings_in_name & ~missing_naics_and_sic) & ~naics_or_sic_correctional].shape
fac[(strings_in_name & ~missing_naics_and_sic) & ~naics_or_sic_correctional].sample(n=10)[industry_code_columns]

(770, 39)

,REGISTRY_ID,PRIMARY_NAME,STATE_NAME,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS
3613383,110034590226,CITY OF ABILENE TERMINAL DRAINAGE IMPROVEMENTS...,TEXAS,NaN,NaN,1794,EXCAVATION WORK
3805541,110033694367,LINDSEY CONSTRUCTION GRAND MISSION REGIONAL DE...,TEXAS,NaN,NaN,1629,"HEAVY CONSTRUCTION, NOT ELSEWHERE CLASSIFIED"
1667612,110041462155,CADDO CORRECTIONAL INSTITUTE LANDFILL,LOUISIANA,562212,SOLID WASTE LANDFILL.,NaN,NaN
3167785,110038433898,LAKE ERIE CORRECTIONAL CENTER,OHIO,NaN,NaN,5541,GASOLINE SERVICE STATIONS
3633978,110043645516,JEFFERSON COUNTY CORRECTIONAL FACILITY,TEXAS,447110,GASOLINE STATIONS WITH CONVENIENCE STORES.,5541,GASOLINE SERVICE STATIONS
1604588,110069471397,BOURBON COUNTY REGIONAL DETENTION CENTER,KENTUCKY,237310,"HIGHWAY, STREET, AND BRIDGE CONSTRUCTION.",1611,"HIGHWAY AND STREET CONSTRUCTION, EXCEPT ELEVAT..."
3793889,110034525850,CITY OF PEARLAND WEST MARYS CREEK DETENTION FA...,TEXAS,NaN,NaN,1629,"HEAVY CONSTRUCTION, NOT ELSEWHERE CLASSIFIED"
695275,110070248910,BOSLEY WASH DETENTION BASIN,COLORADO,NaN,NaN,1799,"SPECIAL TRADE CONTRACTORS, NOT ELSEWHERE CLASS..."
687209,110044876952,CALEY EAST REGIONAL DETENTION POND,COLORADO,NaN,NaN,1799,"SPECIAL TRADE CONTRACTORS, NOT ELSEWHERE CLASS..."
515717,110013853142,"SAN BERN. CO, EPWA COUNTY JAIL",CALIFORNIA,922190,"OTHER JUSTICE, PUBLIC ORDER, AND SAFETY ACTIVI...",NaN,NaN


<a id="latlong"></a>
## Using location data: latitude / longitude
### Notes
In general my notes here are sparse. 

<a href="#top">Back to Top of Notebook</a>

In [63]:
#Column subset to inspect, for convenient reuse
location_columns = ['REGISTRY_ID','PRIMARY_NAME','LATITUDE83','LONGITUDE83','CONVEYOR','COLLECT_DESC','ACCURACY_VALUE','REF_POINT_DESC','SOURCE_DESC']

In [64]:
#visually inspect
fac.sample(n=10)[location_columns]

,REGISTRY_ID,PRIMARY_NAME,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,SOURCE_DESC
172301,110039354604,JOSEPHINE CANYON WWTF,31.564861,-111.003610,AZURITE,UNKNOWN,NaN,NaN,NaN
3817150,110020479007,HOME DEPOT USA HD6547,29.579632,-98.641708,E-PLAN,NaN,NaN,NaN,NaN
427498,110070094040,NEFAB PACKAGING WEST LLC,37.517270,-122.046390,ICIS,NaN,NaN,NaN,NaN
2182540,110003949461,NIXDORF LLOYD CHAIN CO,38.644680,-90.202940,FRS-GEOCODE,ADDRESS MATCHING-HOUSE NUMBER,30.0,ENTRANCE POINT OF A FACILITY OR STATION,NaN
1901142,110052720819,LAKE RESTAURANT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372167,110069993820,RICHIE LAATZ (DBA KJS),NaN,NaN,NaN,NaN,NaN,NaN,NaN
113795,110070515601,CITY OF JOINER MAINTENANCE SHOP,35.508480,-90.150790,FRS-GEOCODE,ADDRESS MATCHING-HOUSE NUMBER,30.0,ENTRANCE POINT OF A FACILITY OR STATION,NaN
61214,110066866993,ARNOLDS AUTO REPAIR,34.673787,-86.043289,FRS-GEOCODE,ADDRESS MATCHING-HOUSE NUMBER,50.0,ENTRANCE POINT OF A FACILITY OR STATION,NaN
3561283,110007845617,CHEMLAWN CORP,35.065657,-85.197764,COMMERCIAL VENDOR,ADDRESS MATCHING-HOUSE NUMBER,150.0,PLANT ENTRANCE (GENERAL),NaN
857315,110070221469,E3Q80: SR30 HIGH TENSION CABLE BARRIER,29.733800,-84.913600,ICIS,NaN,NaN,NaN,NaN


In [27]:
missing_latitude = pd.isnull(fac['LATITUDE83'])
sum(missing_latitude)

missing_longitude = pd.isnull(fac['LONGITUDE83'])
sum(missing_longitude)

missing_long_lat = (missing_longitude & missing_latitude)
sum(missing_long_lat) / fac.shape[0]

1171692

1171692

0.2824763195135887

**Note**: REF_POINT_DESC is the 'what' the location is intended to place (e.g. 'FACILITY CENTROID').

In [29]:
fac[~missing_long_lat]['REF_POINT_DESC'].value_counts(dropna=False).size
fac[~missing_long_lat]['REF_POINT_DESC'].value_counts(dropna=False).head(n=10)

43

CENTER OF A FACILITY OR STATION            1375131
ENTRANCE POINT OF A FACILITY OR STATION     933868
NaN                                         455480
PLANT ENTRANCE (GENERAL)                     49896
POINT WHERE SUBSTANCE IS RELEASED            42303
FACILITY CENTROID                            36753
ACRES POINTS NOT REPRESENTED BY 101-107      31298
UNKNOWN                                      19564
CENTER OF FACILITY                            7941
FACILITY/MONITORING SITE BOUNDARY POINT       6309
Name: REF_POINT_DESC, dtype: int64

In [30]:
fac[naics_or_sic_correctional & ~missing_long_lat]['REF_POINT_DESC'].value_counts(dropna=False).size
fac[naics_or_sic_correctional & ~missing_long_lat]['REF_POINT_DESC'].value_counts(dropna=False)

13

ENTRANCE POINT OF A FACILITY OR STATION                                                                                                                                                                   584
CENTER OF A FACILITY OR STATION                                                                                                                                                                           359
NaN                                                                                                                                                                                                       185
POINT WHERE SUBSTANCE IS RELEASED                                                                                                                                                                          54
PLANT ENTRANCE (GENERAL)                                                                                                                                                        

In [31]:
#string matches in facility name and not coded by NAICS/SIC
fac[(strings_in_name & ~naics_or_sic_correctional) & ~missing_long_lat]['REF_POINT_DESC'].value_counts(dropna=False).size
fac[(strings_in_name & ~naics_or_sic_correctional) & ~missing_long_lat]['REF_POINT_DESC'].value_counts(dropna=False)

18

ENTRANCE POINT OF A FACILITY OR STATION                                                                                                                                                                   743
NaN                                                                                                                                                                                                       629
CENTER OF A FACILITY OR STATION                                                                                                                                                                           457
FACILITY CENTROID                                                                                                                                                                                         103
POINT WHERE SUBSTANCE IS RELEASED                                                                                                                                               

**Note**: COLLECT_DESC is the 'how' the location was determined.

In [32]:
fac[~missing_long_lat]['COLLECT_DESC'].value_counts(dropna=False)

ADDRESS MATCHING-HOUSE NUMBER                                                              2207891
NaN                                                                                         331734
UNKNOWN                                                                                     103553
INTERPOLATION-MAP                                                                            61874
INTERPOLATION-PHOTO                                                                          50966
ADDRESS MATCHING-NEAREST INTERSECTION                                                        46678
ADDRESS MATCHING-BLOCK FACE                                                                  38767
INTERPOLATION-OTHER                                                                          23097
GPS CODE (PSEUDO RANGE) DIFFERENTIAL                                                         20304
GPS - UNSPECIFIED                                                                            15822
ZIP CODE-C

In [33]:
#string matches in facility name and not coded by NAICS/SIC
fac[(strings_in_name & ~naics_or_sic_correctional) & ~missing_long_lat]['COLLECT_DESC'].value_counts(dropna=False).size
fac[(strings_in_name & ~naics_or_sic_correctional) & ~missing_long_lat]['COLLECT_DESC'].value_counts(dropna=False)

25

ADDRESS MATCHING-HOUSE NUMBER                                               1104
NaN                                                                          515
UNKNOWN                                                                      129
INTERPOLATION-MAP                                                            105
ADDRESS MATCHING-NEAREST INTERSECTION                                         46
ADDRESS MATCHING-BLOCK FACE                                                   34
GPS - UNSPECIFIED                                                             28
INTERPOLATION-PHOTO                                                           27
INTERPOLATION-OTHER                                                           15
GPS CODE (PSEUDO RANGE) DIFFERENTIAL                                          10
GPS CODE (PSEUDO RANGE) STANDARD POSITION (SA OFF)                             8
ADDRESS MATCHING-OTHER                                                         7
ZIP CODE-CENTROID           

In [17]:
#string matches in facility name and not coded by NAICS/SIC
fac[strings_in_name & ~naics_or_sic_correctional]['COLLECT_DESC'].value_counts(dropna=False).size
fac[strings_in_name & ~naics_or_sic_correctional]['COLLECT_DESC'].value_counts(dropna=False)

25

NaN                                                                         1300
ADDRESS MATCHING-HOUSE NUMBER                                               1104
UNKNOWN                                                                      129
INTERPOLATION-MAP                                                            105
ADDRESS MATCHING-NEAREST INTERSECTION                                         46
ADDRESS MATCHING-BLOCK FACE                                                   34
GPS - UNSPECIFIED                                                             28
INTERPOLATION-PHOTO                                                           27
INTERPOLATION-OTHER                                                           15
GPS CODE (PSEUDO RANGE) DIFFERENTIAL                                          10
GPS CODE (PSEUDO RANGE) STANDARD POSITION (SA OFF)                             8
ADDRESS MATCHING-OTHER                                                         7
ZIP CODE-CENTROID           

<a id="address"></a>
## Location info: Address

<a href="#top">Back to Top of Notebook</a>

In [42]:
sum(pd.isnull(fac['LOCATION_ADDRESS'])) / fac.shape[0]

0.11280036066182408

In [35]:
fac['LOCATION_ADDRESS'].value_counts(dropna=False)

NaN                                                   467888
UNKNOWN                                                79045
SEE LOCATION DESCRIPTION                               18881
ADDRESS UNKNOWN                                        11971
NO ADDRESS ON RECORD                                    9937
NONE                                                    8404
UNK                                                     6612
NOT AVAILABLE                                           3968
MAIN ST                                                 1742
2608 W MEIGHAN BLVD                                     1481
PORTABLE                                                1450
RR 1                                                    1388
PORTABLE ENGINE- MUST STAY STAND ALONE AT ALL TIME      1273
                                                        1188
PORTABLE SOURCE                                          989
MAIN STREET                                              867
NO STREET ADDRESS       

In [37]:
fac['LOCATION_ADDRESS'].sample(n=10)

1593363       1000 CHESTNUT ST
53324         7600 VAUGHN ROAD
1014413      3826 CLAIRMONT RD
643543     1085 S SANTA FE AVE
876446                 UNKNOWN
3098204           95 SWAIM AVE
2668134      158 WASHINGTON ST
2614964       975 GARFIELD AVE
3940964                    NaN
3009242     16 MERRYS POINT RD
Name: LOCATION_ADDRESS, dtype: object

In [40]:
#coded by NAICS/SIC
fac[naics_or_sic_correctional]['LOCATION_ADDRESS'].value_counts(dropna=False).size
fac[naics_or_sic_correctional]['LOCATION_ADDRESS'].value_counts(dropna=False)

1447

NaN                                                   13
UNKNOWN                                                9
FACILITYWIDE                                           4
846 NE 54TH TERRACE                                    3
3901 KLEIN BLVD                                        3
24414 MUSSELWHITE DRIVE                                2
44750 060TH ST W                                       2
20 MANNING AVE                                         2
100 PRISON RD                                          2
1 FEDERAL WAY                                          2
15801 SW 137TH AVE                                     2
4001 KING                                              2
1600 INDUSTRIAL PARK ROAD                              2
ROUTE 175                                              2
30 HACKENSACK AVE                                      2
US 15                                                  2
2727 HWY K                                             2
1299 SEASIDE AVE               

In [39]:
#string matches in facility name and not coded by NAICS/SIC
fac[strings_in_name & ~naics_or_sic_correctional]['LOCATION_ADDRESS'].value_counts(dropna=False).size
fac[strings_in_name & ~naics_or_sic_correctional]['LOCATION_ADDRESS'].value_counts(dropna=False)

2255

NaN                                                   420
UNKNOWN                                                56
NO ADDRESS ON RECORD                                   16
SEE LOCATION DESCRIPTION                               16
UNK                                                     4
300 PRISON ROAD                                         3
NONE                                                    3
9850 TWIN CITIES RD                                     3
13520 WESTPARK DR                                       3
RT 1 & RAHWAY AVE                                       3
14560 S STATE ST                                        3
1173 FRONT ST                                           3
441 BAUCHET ST                                          2
19005 WILEYS WELL RD                                    2
19 ST AND DALE ST                                       2
NW 10TH ST                                              2
451 RIVERVIEW PKWY                                      2
4913 HELBUSH D

<a id="typeprogram"></a>
## Exploring Facility Type and Program Interest

<a href="#top">Back to Top of Notebook</a>

In [44]:
#of note, a limited number of facility types
fac['SITE_TYPE_NAME'].value_counts(dropna=False)

STATIONARY                       3500397
WATER SYSTEM                      370356
NaN                               145166
FACILITY                           77533
BROWNFIELDS SITE                   28578
MONITORING STATION                 10266
POTENTIALLY CONTAMINATED SITE       7974
PORTABLE                            5469
CONTAMINATED SITE                   1929
CONTAMINATION ADDRESSED              162
MOBILE                                73
WATERFRONT FACILITY                   14
PIPELINE                               8
BARGE                                  5
Name: SITE_TYPE_NAME, dtype: int64

In [45]:
fac['INTEREST_TYPES'].value_counts(dropna=False).size
fac['INTEREST_TYPES'].value_counts(dropna=False).head(n=50) #limit for web viewing

39267

STATE MASTER                                                             1600281
UNSPECIFIED UNIVERSE                                                      293337
TRANSIENT NON-COMMUNITY WATER SYSTEM                                      258201
ICIS-NPDES NON-MAJOR, STORM WATER CONSTRUCTION                            177327
ICIS-NPDES NON-MAJOR                                                      156495
STATE MASTER, UNSPECIFIED UNIVERSE                                        109103
TRANSPORTER                                                                93735
CESQG                                                                      93200
AIR MINOR                                                                  91995
COMMUNITY WATER SYSTEM                                                     79281
CESQG, STATE MASTER                                                        76760
OSHA ESTABLISHMENT                                                         71047
ENFORCEMENT/COMPLIANCE ACTIV

In [46]:
fac[naics_or_sic_correctional]['INTEREST_TYPES'].value_counts(dropna=False).size
fac[naics_or_sic_correctional]['INTEREST_TYPES'].value_counts(dropna=False).head(n=50) #limit for web viewing

458

STATE MASTER                                                                                                       395
ICIS-NPDES NON-MAJOR                                                                                                78
AIR MINOR                                                                                                           75
AIR EMISSIONS CLASSIFICATION UNKNOWN                                                                                71
AIR MINOR, STATE MASTER                                                                                             57
CESQG, STATE MASTER                                                                                                 31
AIR SYNTHETIC MINOR, STATE MASTER                                                                                   22
STATE MASTER, UNSPECIFIED UNIVERSE                                                                                  21
AIR SYNTHETIC MINOR                             

In [47]:
fac[strings_in_name & ~naics_or_sic_correctional]['INTEREST_TYPES'].value_counts(dropna=False).size
fac[strings_in_name & ~naics_or_sic_correctional]['INTEREST_TYPES'].value_counts(dropna=False).head(n=50) #limit for web viewing

174

STATE MASTER                                                           1277
ICIS-NPDES NON-MAJOR, STORM WATER CONSTRUCTION                          285
COMMUNITY WATER SYSTEM                                                  210
ICIS-NPDES NON-MAJOR                                                    206
UNSPECIFIED UNIVERSE                                                     84
CESQG                                                                    51
BROWNFIELDS PROPERTY                                                     46
TRANSPORTER                                                              42
NON-TRANSIENT NON-COMMUNITY WATER SYSTEM                                 41
ICIS-NPDES NON-MAJOR, STATE MASTER, STORM WATER CONSTRUCTION             40
ENFORCEMENT/COMPLIANCE ACTIVITY                                          35
CESQG, STATE MASTER                                                      33
TRANSIENT NON-COMMUNITY WATER SYSTEM                                     29
AIR MINOR   

In [48]:
fac[strings_in_name & ~naics_or_sic_correctional]['LOCATION_DESCRIPTION'].value_counts(dropna=False).size
fac[strings_in_name & ~naics_or_sic_correctional]['LOCATION_DESCRIPTION'].value_counts(dropna=False).head(n=50) #limit for web viewing      

131

NaN                                                                                           2634
                                                                                                65
NEW RST; RST CONVERSION PROJECT 05/15/2005                                                       7
UNDERGROUND STORAGE TANK                                                                         5
ELOY                                                                                             5
FIRST POINT CLICKED IN WUSE MAP APPLET                                                           2
NM-TEMPO                                                                                         2
ALABAMA DEPARTMENT OF CORRECTIONS                                                                2
FROM 363 TO MELALEVCA DR                                                                         1
PHOENIX                                                                                          1
CORRECTION